## Importar librerias

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Noticias_BCRP/notebooks/")

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
from gensim.models import word2vec
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm,model_selection
from sklearn.cluster import KMeans

In [0]:
from sklearn.linear_model import Lasso
model = Lasso()
model.fit()

In [0]:
modelo=word2vec.Word2Vec()
modelo.window

In [0]:
import sys
### importar libreriar locales
sys.path.append("/content/drive/My Drive/Colab Notebooks/Noticias_BCRP/src")
from plot_word2vec import plot_word2vec
from vectorization_word2vec_boew import *

## Cargar train y test

In [0]:
df_train=pd.read_pickle("../data/preprocessed/df_train.pkl")
df_test=pd.read_pickle("../data/preprocessed/df_test.pkl")

In [0]:
y_train=df_train["Target"]
y_test=df_test["Target"]

In [0]:
news_train=[ i.split() for i in df_train["Clean_Stopword_Info"]]
news_test=[ i.split() for i in df_test["Clean_Stopword_Info"]]

## Vectorizacion de noticias - Boew

In [0]:
try:
  modelo=pickle.load(open("../models/boew/modelo.pkl","rb"))
except:
  ## Preparando el corpus para el entrenamiento del Word2Vec
  archivo=open(file='../models/boew/models.txt',mode='w',encoding="utf 8")  
  for elemento in news_train:
      strg=" ".join(elemento)
      archivo.write(strg +"\n")
  archivo.close()
  ## Construccion del modelo word2vec 
  sentences =word2vec.Text8Corpus('../models/boew/models.txt')
  modelo = word2vec.Word2Vec(sentences,size=100,min_count=5)
  len(modelo.wv.vocab)
  ### guardar el modelo construido de word2vec
  pickle.dump(modelo,open("../models/boew/modelo.pkl","wb"))

In [0]:
k_es=[10,15,20,25,30,35,40,45,50]
for k in k_es:
  ### entrenar un kmeans
  [kmeans_model,scaler_kmeans,vector]=kmeans_word2vec(modelo,k)
  ### obtener la vectorizacion boew
  boew_train=vectorization_boew(news_train,modelo,scaler_kmeans,kmeans_model,k)
  boew_test=vectorization_boew(news_test,modelo,scaler_kmeans,kmeans_model,k)
  ### estandarizar los valores
  boew_train=np.asarray(boew_train)
  boew_test=np.asarray(boew_test)
  scaler_boew = StandardScaler() 
  scaler_boew.fit(boew_train)###obtiene el valor de la media y la desviacion estandar
  boew_train = scaler_boew.transform(boew_train) ### normalizar los datos de entrada para el entrenamiento
  boew_test = scaler_boew.transform(boew_test)   ### normalizar los datos de entrada para la prueba
  print(boew_train.shape,boew_test.shape)
  ### guardar los modelos
  pickle.dump(scaler_kmeans,open("../models/boew/scaler_kmeans_{}.pkl".format(k),"wb"))
  pickle.dump(kmeans_model,open("../models/boew/kmeans_model_{}.pkl".format(k),"wb"))
  pickle.dump(scaler_boew,open("../models/boew/scaler_boew_{}.pkl".format(k),"wb"))
  ### Guarda la vectorizacion word2vec
  np.save("../data/preprocessed/boew/boew_train_{}.npy".format(k),boew_train)
  np.save("../data/preprocessed/boew/boew_test_{}.npy".format(k),boew_test)

(48957, 20) (12240, 20)
(48957, 25) (12240, 25)
(48957, 30) (12240, 30)
(48957, 35) (12240, 35)
(48957, 40) (12240, 40)
(48957, 45) (12240, 45)
(48957, 50) (12240, 50)
